# How does **GROUPBY** happen in **DASK** (distributed framework)?

<!-- ![title](img/panda.png) -->
<img src="img/panda.png" width=800 />

In [3]:
import pandas as pd
import dask.dataframe as dd

In [4]:
pdf = pd.DataFrame(dict(a=[1, 1, 2, 3, 3, 1, 1, 2, 3, 3, 4, 4, 1, 3, 6],b=[1, 3, 10, 3, 2, 1, 3, 10, 3, 3, 12, 0, 9, 2, 4],c=[2, 4, 5, 2, 3, 5, 2, 3, 9, 2, 44, 33, 2, 4,1]))
pdf

,a,b,c
0,1,1,2
1,1,3,4
2,2,10,5
3,3,3,2
4,3,2,3
5,1,1,5
6,1,3,2
7,2,10,3
8,3,3,9
9,3,3,2


In [5]:
ddf = dd.from_pandas(pdf, npartitions=3)
ddf

,a,b,c
npartitions=3,,,
0,int64,int64,int64
5,...,...,...
10,...,...,...
14,...,...,...


In [6]:
ddf.partitions[2].compute()

,a,b,c
10,4,12,44
11,4,0,33
12,1,9,2
13,3,2,4
14,6,4,1


In [7]:
sdf1 = pdf[:5]
sdf2 = pdf[5:10]
sdf3 = pdf[10:]

In [8]:
gp1 = sdf1.groupby(['a', 'b']).c.sum()
gp2 = sdf2.groupby(['a', 'b']).c.sum()
gp3 = sdf3.groupby(['a', 'b']).c.sum()

In [9]:
gp1,gp2,gp3

(a  b 
 1  1     2
    3     4
 2  10    5
 3  2     3
    3     2
 Name: c, dtype: int64,
 a  b 
 1  1      5
    3      2
 2  10     3
 3  3     11
 Name: c, dtype: int64,
 a  b 
 1  9      2
 3  2      4
 4  0     33
    12    44
 6  4      1
 Name: c, dtype: int64)

In [10]:
concated_gp = pd.concat([gp1, gp2, gp3])

In [11]:
concated_gp

a  b 
1  1      2
   3      4
2  10     5
3  2      3
   3      2
1  1      5
   3      2
2  10     3
3  3     11
1  9      2
3  2      4
4  0     33
   12    44
6  4      1
Name: c, dtype: int64

In [12]:
total = concated_gp.groupby(level=[0, 1]).sum()

In [13]:
total

a  b 
1  1      7
   3      6
   9      2
2  10     8
3  2      7
   3     13
4  0     33
   12    44
6  4      1
Name: c, dtype: int64

In [14]:
pdf.groupby(['a', 'b']).c.sum()

a  b 
1  1      7
   3      6
   9      2
2  10     8
3  2      7
   3     13
4  0     33
   12    44
6  4      1
Name: c, dtype: int64

In [15]:
ddf.groupby(['a', 'b']).c.sum().compute()

a  b 
1  1      7
   3      6
2  10     8
3  2      7
   3     13
1  9      2
4  0     33
   12    44
6  4      1
Name: c, dtype: int64

## **Dask Delayed GROUBY**

In [3]:
from dask import delayed
import pandas as pd

In [28]:
from dask.distributed import Client

client = Client(threads_per_worker=4)

client

/home/koustav/Documents/DataframeWar/env/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36283 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:33035 Dashboard: http://127.0.0.1:36283/status,Cluster Workers: 3 Cores: 12 Memory: 8.18 GB


In [16]:
df = delayed(pd.read_csv)('../Data/beer_small.csv')

In [5]:
df

Delayed('read_csv-4e4fcf69-a3f0-44f8-a867-edf7f8d2339a')

In [7]:
columns = df.columns

In [8]:
columns.compute()

Index(['Unnamed: 0', 'brewery_id', 'brewery_name', 'review_time',
       'review_overall', 'review_aroma', 'review_appearance',
       'review_profilename', 'beer_style', 'review_palate', 'review_taste',
       'beer_name', 'beer_abv', 'beer_beerid'],
      dtype='object')

In [9]:
df.head()

Delayed('head-06e8de6a-2937-4fe2-8294-a1cd2789b85e')

In [10]:
df.head().compute()

,Unnamed: 0,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,784200,952,Great Dane Pub & Brewing Company (Downtown),1136269921,4.5,4.0,4.0,dirtylou,American IPA,4.0,4.0,Texas Speedbump IPA,NaN,11846
1,1305265,29,Anheuser-Busch,1234830966,4.5,4.0,3.0,talkinghatrack,Light Lager,3.0,4.0,Bud Light Lime,4.2,41821
2,1526298,45,Brooklyn Brewery,1078599557,4.5,4.0,4.0,PopeJonPaul,Scotch Ale / Wee Heavy,4.0,4.5,Brooklyn Heavy Scotch Ale,7.5,16355
3,450647,590,New Glarus Brewing Company,1288790879,4.5,4.5,4.5,sweemzander,American Wild Ale,4.5,4.0,R&D Bourbon Barrel Kriek,5.5,60588
4,1223094,4,Allagash Brewing Company,1295320417,4.5,4.5,4.0,Jmoore50,American Wild Ale,4.0,4.0,Allagash Victor Francenstein,9.7,56665


In [17]:
ipa = df[df.beer_style.str.contains('IPA')]

In [23]:
mean_ipa_review = ipa.groupby('brewery_name').review_overall.agg(['mean','count'])

In [24]:
mean_ipa_review

Delayed('agg-9dcdae6a-0187-47c8-accb-929608674f42')

In [25]:
mean_ipa_review.compute()

,mean,count
brewery_name,,
(512) Brewing Company,3.785714,7
10 Barrel Brewing Co.,4.000000,1
1516 Brewing Company,4.000000,1
16 Mile Brewing Company,3.500000,1
1702 / The Address Brewing Co.,4.000000,1
...,...,...
barVolo,4.000000,1
the Nils Oscar Company,3.850000,10
À La Fût,4.500000,1


In [29]:
df2 = delayed(pd.read_csv)("../Data/yellow_tripdata_2020-01.csv")

In [32]:
df2.head().compute()

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


KilledWorker: ('read_csv-314d07a0-5472-47ec-83a7-c9027bbc287e', <Worker 'tcp://127.0.0.1:43003', name: 1, memory: 0, processing: 1>)